In [34]:
from tensorflow.keras.layers import TextVectorization
import tensorflow as tf
import io
import numpy as np
import pickle

In [35]:
def read_corpus(corpus_file):
    '''Read in data set and returns docs and labels'''
    documents = []
    labels = []
    with open(corpus_file, encoding='utf-8') as f:
        for line in f:
            tokens = line.strip()
            documents.append("".join(tokens.split("\t")[0]).strip())
            # binary problem: NOT, OFF
            labels.append(tokens.split("\t")[1])
    return documents, labels

In [36]:
#Read in the data
X_train, Y_train = read_corpus("../datasets/train_preprocessed.txt")
X_dev, Y_dev = read_corpus("../datasets/val_preprocessed.txt")
# Transform words to indices using a vectorizer
vectorizer = TextVectorization(standardize=None)
# Use train and dev to create vocab - could also do just train
text_ds = tf.data.Dataset.from_tensor_slices(X_train + X_dev)
with tf.device('/cpu:0'):
    vectorizer.adapt(text_ds)
# Dictionary mapping words to idx
voc = vectorizer.get_vocabulary()

2022-10-30 03:27:33.375781: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


KeyboardInterrupt: 

In [ ]:
#fasttext embeddings filter and save
emb_file="crawl-300d-2M-subword.vec"
fin = io.open(emb_file, 'r', encoding='utf-8', newline='\n', errors='ignore')
n, d = map(int, fin.readline().split())
data = {}
for line in fin:
    tokens = line.rstrip().split(' ')
    if tokens[0] in voc:
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
with open('fasttext.pkl', 'wb') as f:
    pickle.dump(data, f)

In [19]:
#glove embeddings 25d, 50d, 100d, 200d filter and save
emb_files=["glove.twitter.27B.25d.txt","glove.twitter.27B.50d.txt","glove.twitter.27B.100d.txt","glove.twitter.27B.200d.txt"]
output_files=["glove_25d.pkl","glove_50d.pkl","glove_100d.pkl","glove_200d.pkl"]

for i in range(len(emb_files)):
    fin = io.open(emb_files[i], 'r', encoding='utf-8', newline='\n', errors='ignore')
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        if tokens[0] in voc:
            data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    with open(output_files[i], 'wb') as f:
        pickle.dump(data, f)